# Let's get this party started

In [1]:
#Dependencies
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime as dt

# Framing the Problem & Big Picture look (see readme file)

# Get the Data

In [2]:
#set path to datasets
path = os.getcwd()
os.chdir("../datasets/")
filename = "sampledata.xlsx"
ser_path = "./serialized/"
filetwo = "sale_orders.xlsx"
filethree = "new_inventory.xlsx"

In [3]:
#Importing company datasets
#prod_hist = pd.read_excel(filename, sheet_name="production_hist")
#ship_hist = pd.read_excel(filename, sheet_name="shipment_hist")
#atp_hist = pd.read_excel(filename, sheet_name="atp_hist")
#inventory = pd.read_excel(filename, sheet_name="inventory")

In [4]:
#importing sales orders and new inventory
# order_hist = pd.read_excel(filetwo, sheet_name = "sale_orders")
# inv_hist2 = pd.read_excel(filethree, sheet_name = "new_inv")

In [5]:
# #setting file names
# data = [inventory, atp_hist, prod_hist, ship_hist]
# fnames = ["inventory", "atp_hist", "prod_hist","ship_hist"]
# count = 0
# for df in data:
#     df.name = fnames[count]
#     count+=1

In [6]:
# #saving dataframes into pickle files
# for df in data:
#     pklfile = df.name + ".pkl"
#     df.to_pickle(ser_path + pklfile)

In [7]:
# #setting file names
# data2 = [order_hist, inv_hist2]
# fnames = ["order_hist", "inv_hist2"]

# count = 0
# for df in data2:
#     df.name = fnames[count]
#     count+=1

# #saving new data into pickle files
# for df in data2:
#     pklfile = df.name + ".pkl"
#     df.to_pickle(ser_path + pklfile)

#saving dataframes into feather (not working)
for df in data:
    feafile = df.name
    df.to_feather(ser_path+feafile)

In [8]:
#see pklfile name order
pklfiles = os.listdir("./serialized")
pklfiles

['ship_hist.pkl',
 'inventory.pkl',
 'inv_hist2.pkl',
 'order_hist.pkl',
 'atp_hist.pkl',
 'prod_hist.pkl',
 'merged.pkl']

In [147]:
#Fast Upload using pickle files
ship_hist = pd.read_pickle(ser_path+pklfiles[0])
inventory = pd.read_pickle(ser_path+pklfiles[1])
inv_hist2 = pd.read_pickle(ser_path+pklfiles[2])
order_hist = pd.read_pickle(ser_path+pklfiles[3])
atp_hist = pd.read_pickle(ser_path+pklfiles[4])
prod_hist = pd.read_pickle(ser_path+pklfiles[5])
merged_df = pd.read_pickle(ser_path+pklfiles[6])

# Cleaning Up

### function clean_columns

In [10]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["requested_date"] = dataframe["requested_date"].astype("datetime64")
        dataframe["confirmed_date"] = dataframe["confirmed_date"].astype("datetime64")
        dataframe["material_avail_date"] = dataframe["material_avail_date"].astype("datetime64")
        dataframe["load_date"] = dataframe["load_date"].astype("datetime64")
        dataframe["plan_goods_issue_date"] = dataframe["plan_goods_issue_date"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [11]:
# attempt to get rid of nulls, need to be more specific with fills to not loose valuable insight. 
# data = [inventory, atp_hist, prod_hist, ship_hist]
# for df in data:
#     df = df.fillna(0, inplace=True)

### Inventory

In [12]:
clean_columns(inventory)

Index(['calendar_day', 'plant', 'material', 'maktx', 'ph3', 'ph5', 'ph',
       'material_type', 'tons', 'shape', 'size_for_qual_cert', 'strgr',
       'block_resource', 'work_center', 'atp'],
      dtype='object')

In [13]:
inventory.head(5)

,calendar_day,plant,material,maktx,ph3,ph5,ph,material_type,tons,shape,size_for_qual_cert,strgr,block_resource,work_center,atp
0,2020-02-23,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
1,2020-02-20,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
2,2020-02-19,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
3,2020-02-18,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN
4,2020-02-17,1302,106022474,"L 2X2X1/4 A36/44W/A529-50 20'00""",LS-Mills-Mer,LS-Mills-Mer-Mer-Lght Shp,LS-Mills-Mer-Mer-Lght Shp-MBQ Angles,ZERT,0.009,SHAPE_L,2X2X1/4,63,"2"" ANG",25000001.0,NaN


In [14]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850873 entries, 0 to 850872
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   calendar_day        850873 non-null  datetime64[ns]
 1   plant               850873 non-null  int64         
 2   material            850873 non-null  int64         
 3   maktx               850873 non-null  object        
 4   ph3                 850873 non-null  object        
 5   ph5                 850873 non-null  object        
 6   ph                  850873 non-null  object        
 7   material_type       850873 non-null  object        
 8   tons                850873 non-null  float64       
 9   shape               848401 non-null  object        
 10  size_for_qual_cert  848401 non-null  object        
 11  strgr               850873 non-null  int64         
 12  block_resource      626217 non-null  object        
 13  work_center         850866 no

In [15]:
#Filling in empty atp rows with zero
inventory["atp"].fillna(0, inplace=True)

In [16]:
# inventory["work_center"] = inventory["work_center"].astype("int64")

In [17]:
inventory = inventory.loc[inventory["material_type"]=="ZERT"]
inventory["material_type"].unique()

array(['ZERT'], dtype=object)

In [18]:
inv_reduced = inventory[["calendar_day", "strgr", "material", "maktx", "work_center", "ph", "shape", "size_for_qual_cert", "block_resource", "tons", "atp"]]
inv_reduced.rename(columns={"material":"sku", "maktx":"material_description", "size_for_qual_cert":"size", "block_resource":"block", "strgr":"strategy"}, inplace=True)
inv_reduced = inv_reduced.sort_values(["calendar_day", "material_description"])
inv_reduced.head()

,calendar_day,strategy,sku,material_description,work_center,ph,shape,size,block,tons,atp
799352,2017-01-01,63,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,9.442000,0.0
799094,2017-01-01,63,102003053,"BL 5 1/8X7 3/8 1010M-446_1302 3,858 SA",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,1.929000,0.0
799267,2017-01-01,63,102003353,BL 5 1/8X7 3/8 GAU_1050-501_1302 3168 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,7.920000,0.0
799180,2017-01-01,63,102003354,BL 5 1/8X7 3/8 GAU_MG12-500_1302 3220 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,11.380000,0.0
844399,2017-01-01,40,106020238,"C 10X15.3# GGMULTI 20'00""",25000001.0,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,10 X 15.3#,"10"" CHN",110.007004,0.0


In [19]:
#changing dtype
inv_reduced["strategy"] = inv_reduced["strategy"].astype("category")
inv_reduced["sku"] = inv_reduced["sku"].astype("object")
inv_reduced["work_center"] = inv_reduced["work_center"].astype("category")

In [20]:
#view of new dtypes & counts
inv_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649986 entries, 799352 to 18367
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   calendar_day          649986 non-null  datetime64[ns]
 1   strategy              649986 non-null  category      
 2   sku                   649986 non-null  object        
 3   material_description  649986 non-null  object        
 4   work_center           649979 non-null  category      
 5   ph                    649986 non-null  object        
 6   shape                 647514 non-null  object        
 7   size                  647514 non-null  object        
 8   block                 626217 non-null  object        
 9   tons                  649986 non-null  float64       
 10  atp                   649986 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(2), object(6)
memory usage: 50.8+ MB


In [21]:
#peak at all nulls across
inv_reduced[inv_reduced.isna().any(axis=1)]

,calendar_day,strategy,sku,material_description,work_center,ph,shape,size,block,tons,atp
799352,2017-01-01,63,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,9.442,0.0
799094,2017-01-01,63,102003053,"BL 5 1/8X7 3/8 1010M-446_1302 3,858 SA",18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,1.929,0.0
799267,2017-01-01,63,102003353,BL 5 1/8X7 3/8 GAU_1050-501_1302 3168 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,7.920,0.0
799180,2017-01-01,63,102003354,BL 5 1/8X7 3/8 GAU_MG12-500_1302 3220 SA,18000001.0,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,5 1/8 X 7 3/8,NaN,11.380,0.0
840484,2017-01-01,40,106020149,"C 12X30# GGMULTI 20'00""",25000001.0,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,12 X 30#,NaN,1.200,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3819,2020-02-23,40,109004211,"PR 3/8"" 1018 20'00"" CD",25000001.0,LS-Mills-Mer-Mer-MBQ-Pencil Rod,PENCIL_ROD,"3/8""",NaN,-0.002,0.0
19798,2020-02-23,63,205012261,"RD .678"" 1055MC 31'00""",25000002.0,SS-Mills-SBQ-SBQ-SBQ-SBQ Rd,ROUND,".678""",NaN,-0.001,0.0
8136,2020-02-23,40,109250351,"RD 1 1/4"" GGMULTI 20'00""",25000002.0,LS-Mills-Mer-Mer-MBQ-MBQ Rd,ROUND,"1 1/4""",NaN,2.529,0.0
8134,2020-02-23,63,109250353,"RD 1/2"" GGMULTI 20'00""",25000002.0,LS-Mills-Mer-Mer-MBQ-MBQ Rd,ROUND,"1/2""",NaN,-0.002,0.0


### ATP History

In [22]:
clean_columns(atp_hist)

Index(['plant', 'plant_description', 'sku', 'atp_floor', 'snapshot_date'], dtype='object')

In [23]:
atp_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235706 entries, 0 to 235705
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   plant              235706 non-null  int64         
 1   plant_description  235706 non-null  object        
 2   sku                235706 non-null  object        
 3   atp_floor          235706 non-null  float64       
 4   snapshot_date      235706 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 9.0+ MB


In [24]:
atp_reduced = atp_hist[["sku", "atp_floor", "snapshot_date"]]
atp_reduced = atp_reduced.sort_values(["snapshot_date", "sku"])
atp_reduced.head()

,sku,atp_floor,snapshot_date
109514,101500984,12.306,2017-12-04
109513,102003236,9.442,2017-12-04
109512,102003353,1.584,2017-12-04
18494,102003354,2.380,2017-12-04
109511,102508509,17.590,2017-12-04


In [25]:
atp_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235706 entries, 109514 to 132048
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   sku            235706 non-null  object        
 1   atp_floor      235706 non-null  float64       
 2   snapshot_date  235706 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 7.2+ MB


### Production History

In [26]:
clean_columns(prod_hist)

Index(['calendar_day', 'plant', 'plant_description', 'production_process',
       'production_process_description', 'work_center', 'wc_description',
       'material', 'material_description', 'ph3', 'ph5', 'ton', 'block',
       'shape', 'size', 'stratedgy', 'month/year', 'year', 'monthyear'],
      dtype='object')

In [27]:
prod_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41935 entries, 0 to 41934
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   calendar_day                    41935 non-null  datetime64[ns]
 1   plant                           41935 non-null  int64         
 2   plant_description               41935 non-null  object        
 3   production_process              41935 non-null  int64         
 4   production_process_description  41935 non-null  object        
 5   work_center                     37833 non-null  object        
 6   wc_description                  41935 non-null  object        
 7   material                        41935 non-null  int64         
 8   material_description            41935 non-null  object        
 9   ph3                             36579 non-null  object        
 10  ph5                             36579 non-null  object        
 11  to

In [28]:
# prod_hist["stratedgy"] = prod_hist["stratedgy"].astype("int64")

In [29]:
prod_reduced = prod_hist[["calendar_day", "production_process", "production_process_description","stratedgy","work_center","wc_description","block","shape","size","material","material_description","ton"]]
prod_reduced.rename(columns={"material":"sku", "stratedgy":"strategy"}, inplace=True)
prod_reduced = prod_reduced.sort_values(["calendar_day", "material_description"])
prod_reduced.head()

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
39204,2013-02-01,4306,Scrap yard,NaN,17200001,1302/17200001,NaN,NaN,NaN,56000099,INDUSTRIAL - #1 HM,58
41067,2013-02-01,4306,Scrap yard,NaN,17200001,1302/17200001,NaN,NaN,NaN,56000093,INDUSTRIAL - #2 SHRED,359
30631,2013-02-04,4307,Melt shop,10.0,19000001,MELT SHOP - CONTINUOUS CASTING,NaN,RECTANGULAR,5 1/8 X 7 3/8,102508981,"BL 5 1/8X7 3/8 TRIPLB-393_1302 3,662",114
22721,2013-02-04,4307,Melt shop,10.0,19000001,MELT SHOP - CONTINUOUS CASTING,NaN,RECTANGULAR,5 1/8 X 7 3/8,102508832,"BL 5 1/8X7 3/8 TRIPLB-424_1302 4,002",42
30634,2013-02-04,4307,Melt shop,10.0,19000001,MELT SHOP - CONTINUOUS CASTING,NaN,RECTANGULAR,5 1/8 X 7 3/8,102508836,"BL 5 1/8X7 3/8 TRIPLB-426_1302 3,516",791


In [30]:
#Removing strategy 10 (Note: check assumption that these are all billets)
prod_reduced = prod_reduced.loc[(prod_reduced["strategy"] !=10)]

In [31]:
#Removing prod_processes up the supply chain: melt, scrap etc.
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Melt shop") & (prod_reduced["production_process_description"] != "Scrap yard")]
prod_reduced = prod_reduced.loc[(prod_reduced["production_process_description"] != "Finishing production")]

In [32]:
#Identifying strategy=NaN rows
prod_reduced[prod_reduced["strategy"].isna()]

,calendar_day,production_process,production_process_description,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
36154,2015-02-27,4310,Rolling,NaN,25000002,ROLLING - BAR,NaN,NaN,NaN,40000148,COPROD - FG YIELD LOSS (ROLLING MILL),10
36155,2016-12-29,4310,Rolling,NaN,25000002,ROLLING - BAR,NaN,NaN,NaN,40000148,COPROD - FG YIELD LOSS (ROLLING MILL),-10


In [33]:
#Identifying size=NaN rows
#prod_reduced[prod_reduced["size"].isna()]

In [34]:
#Dropping selective NaN rows
prod_reduced.dropna(subset=["strategy"], inplace=True)

In [35]:
#Identifying unique inputs by column
unique_counts = pd.DataFrame.from_records([(col, prod_reduced[col].nunique()) for col in prod_reduced.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

,Column_Name,Num_Unique
1,production_process,1
2,production_process_description,1
3,strategy,2
4,work_center,4
5,wc_description,4
7,shape,6
6,block,41
8,size,175
11,ton,1299
9,sku,1862


In [36]:
# Changing data types for colums, 
for col in prod_reduced.columns:
    if prod_reduced[col].nunique() < 10:
        prod_reduced[col] = prod_reduced[col].astype("category")
# not including "block" as it used as object elsewhere

prod_reduced["sku"] =prod_reduced["sku"].astype("object")

In [37]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14820 entries, 192 to 13808
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   calendar_day                    14820 non-null  datetime64[ns]
 1   production_process              14820 non-null  category      
 2   production_process_description  14820 non-null  category      
 3   strategy                        14820 non-null  category      
 4   work_center                     13450 non-null  category      
 5   wc_description                  14820 non-null  category      
 6   block                           14815 non-null  object        
 7   shape                           14795 non-null  category      
 8   size                            14795 non-null  object        
 9   sku                             14820 non-null  object        
 10  material_description            14820 non-null  object        
 11  

In [38]:
prod_reduced[prod_reduced["block"].isna()]["production_process_description"]

36135    Rolling
35617    Rolling
35618    Rolling
35632    Rolling
35631    Rolling
Name: production_process_description, dtype: category
Categories (1, object): [Rolling]

In [39]:
prod_reduced[prod_reduced["block"].isna()].groupby("production_process_description").count()

,calendar_day,production_process,strategy,work_center,wc_description,block,shape,size,sku,material_description,ton
production_process_description,,,,,,,,,,,
Rolling,5,5,5,3,5,0,5,5,5,5,5


#### Adding WC_dictionary

In [40]:
#needs work
#prod_reduced.groupby(["PR_sku","PR_wc_description"]).sum().reset_index()

#work_cent_dict = 
#size_dict = size_dict.reset_index()
#size_dict = size_dict.drop("tons", axis=1)

### Ship History

In [41]:
clean_columns(ship_hist)

Index(['shipping_day', 'plant', 'plant_description', 'sales_organization',
       'sales_doc_type', 'outside_rep_ship-to', 'inside_rep_ship-to',
       'sold-to_party', 'sold-to_part_name', 'ship-to_party',
       'ship-to_party_name', 'postal_code', 'location', 'region',
       'region_description', 'country', 'country_description', 'material',
       'material_description', 'div_id', 'div', 'prod_hier_level_3',
       'prod_hier_level_5', 'shipment_tons', 'manager', 'segment',
       'plantcountry', 'businessdaycounter', 'block', 'lvorm', 'mrptype',
       'proctype', 'relativemonth', 'shipping_week', 'strgr', 'shape',
       'sizeforqualcert', 'dailyshiprate3', 'monthdelta', 'dailyshiprate6',
       'dailyshiprate12', 'workcenter', 'wc_desc', 'bayou_flag'],
      dtype='object')

In [42]:
ship_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156796 entries, 0 to 156795
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   shipping_day          156796 non-null  datetime64[ns]
 1   plant                 156796 non-null  int64         
 2   plant_description     156796 non-null  object        
 3   sales_organization    156796 non-null  object        
 4   sales_doc_type        156796 non-null  object        
 5   outside_rep_ship-to   156796 non-null  object        
 6   inside_rep_ship-to    156796 non-null  object        
 7   sold-to_party         156796 non-null  int64         
 8   sold-to_part_name     156796 non-null  object        
 9   ship-to_party         156796 non-null  int64         
 10  ship-to_party_name    156796 non-null  object        
 11  postal_code           156796 non-null  object        
 12  location              156796 non-null  object        
 13 

In [43]:
#Filling in NaN Shipment field rows to zero
ship_hist["shipment_tons"].fillna(0, inplace=True)

In [44]:
# Attempt to get correct data types, needs work.
# ship_hist["businessdaycounter"] = ship_hist["businessdaycounter"].astype("int64")
# ship_hist["strgr"] = ship_hist["strgr"].astype("int64")
# ship_hist["workcenter"] = ship_hist["workcenter"].astype("int64")

In [45]:
#Reducing main data frame
ship_reduced = ship_hist[["shipping_day", "block", "strgr", "shape", "sizeforqualcert", "div", "material", "material_description","shipment_tons", "segment", "sold-to_party","ship-to_party", "postal_code","region_description", "country"]]
ship_reduced.rename(columns={"material":"sku", "strgr":"strategy","sizeforqualcert":"size"}, inplace=True)
ship_reduced = ship_reduced.sort_values(["shipping_day", "material_description"])
ship_reduced.head()

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
155274,2015-01-02,"8"" CHN",40.0,SHAPE_C,8 X 11.5#,Merchants,106021714,"C 8X11.5# GGMULTI 20'00""",6.0,NA - DISTRIBUTION,100064074,100095278,05602-4480,Vermont,US
142630,2015-01-02,"3"" ANG",63.0,SHAPE_L,3X3X3/8,Merchants,106023082,"L 3X3X3/8 55W 60'00""",66.0,NA - CONSTRUCTION,100092191,100095442,J4G 1E5,Quebec,CA
142631,2015-01-02,"3"" ANG",63.0,SHAPE_L,3X3X5/16,Merchants,106023056,"L 3X3X5/16 55W 60'00""",218.0,NA - CONSTRUCTION,100092191,100095442,J4G 1E5,Quebec,CA
153402,2015-01-02,"5"" ANG",40.0,SHAPE_L,5X5X1/2,Merchants,106029552,"L 5X5X1/2 GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100064074,100095278,05602-4480,Vermont,US
153404,2015-01-02,"5"" ANG",40.0,SHAPE_L,5X5X3/8,Merchants,106029532,"L 5X5X3/8 GGMULTI 40'00""",5.0,NA - DISTRIBUTION,100064074,100095278,05602-4480,Vermont,US


In [46]:
#changing sku to object dtype
ship_reduced["sku"] = ship_reduced["sku"].astype("object")
ship_reduced["sold-to_party"] = ship_reduced["sold-to_party"].astype("object")
ship_reduced["ship-to_party"] = ship_reduced["ship-to_party"].astype("object")
ship_reduced["strategy"] = ship_reduced["strategy"].astype("object")

In [47]:
#Check unique Divisions
ship_hist["div"].unique()

array(['Merchants', 'Rebar', 'Billets', 'SBQ Bar', 'Scrap'], dtype=object)

In [48]:
#Removing billets and scrap from dataframe
ship_reduced = ship_reduced.loc[(ship_reduced["div"] != "Billets") & (ship_reduced["div"] != "Scrap")]

In [49]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156711 entries, 155274 to 21756
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   shipping_day          156711 non-null  datetime64[ns]
 1   block                 153079 non-null  object        
 2   strategy              156700 non-null  object        
 3   shape                 156700 non-null  object        
 4   size                  156700 non-null  object        
 5   div                   156711 non-null  object        
 6   sku                   156711 non-null  object        
 7   material_description  156711 non-null  object        
 8   shipment_tons         156711 non-null  float64       
 9   segment               154025 non-null  object        
 10  sold-to_party         156711 non-null  object        
 11  ship-to_party         156711 non-null  object        
 12  postal_code           156711 non-null  object        


In [50]:
#Show Sample of NaN fields
ship_reduced[ship_reduced.isna().any(axis=1)]

,shipping_day,block,strategy,shape,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
99068,2015-01-05,NaN,40,FLAT,5/16 X 8,Merchants,109004173,"F 5/16X8 GGMULTI 20'00""",7.0,NA - DISTRIBUTION,100061219,100061242,K1G 3W3,Ontario,CA
146066,2015-01-05,NaN,40,FLAT,5/8 X 9,Merchants,109004228,"F 5/8X9 GGMULTI 20'00""",4.0,NA - DISTRIBUTION,100060244,100060255,48876-8795,Michigan,US
127810,2015-01-06,NaN,40,FLAT,1/4 X 1,Merchants,109005227,"F 1/4X1 A36/44W 20'00""",3.0,NA - DISTRIBUTION,100092155,100095353,T2C 4W3,Alberta,CA
127807,2015-01-06,NaN,40,FLAT,1/4 X 2,Merchants,109009070,"F 1/4X2 GGMULTI 20'00""",5.0,NA - DISTRIBUTION,100092155,100095353,T2C 4W3,Alberta,CA
127806,2015-01-06,NaN,40,FLAT,1/4 X 3,Merchants,109009081,"F 1/4X3 GGMULTI 20'00""",10.0,NA - DISTRIBUTION,100092155,100095353,T2C 4W3,Alberta,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18290,2020-02-21,"4 X 3"" ANG",40,SHAPE_L,4X3X1/4,Merchants,106028496,"L 4X3X1/4 GGMULTI 20'00""",3.0,NaN,100092138,100095277,V3N 2T8,British Columbia,CA
24479,2020-02-21,"4"" ANG",40,SHAPE_L,4X4X1/4,Merchants,106028759,"L 4X4X1/4 GGMULTI 40'00""",10.0,NaN,100061349,100061360,T2C 2A1,Alberta,CA
24480,2020-02-21,"4"" ANG",40,SHAPE_L,4X4X3/8,Merchants,106028625,"L 4X4X3/8 GGMULTI 40'00""",10.0,NaN,100061349,100061360,T2C 2A1,Alberta,CA
18281,2020-02-21,"5 X 3"" ANG",40,SHAPE_L,5X3X3/8,Merchants,106029499,"L 5X3X3/8 GGMULTI 20'00""",3.0,NaN,100092142,100095283,V4X 2M9,British Columbia,CA


In [51]:
#Show the shape that show as null within the block
ship_reduced[ship_reduced["block"].isna()].groupby("shape").count()

,shipping_day,block,strategy,size,div,sku,material_description,shipment_tons,segment,sold-to_party,ship-to_party,postal_code,region_description,country
shape,,,,,,,,,,,,,,
ASDRAWING,2,0,2,2,2,2,2,2,2,2,2,2,2,2
FLAT,2457,0,2457,2457,2457,2457,2457,2457,2249,2457,2457,2457,2457,2457
PENCIL_ROD,81,0,81,81,81,81,81,81,81,81,81,81,81,81
ROUND,246,0,246,246,246,246,246,246,216,246,246,246,246,246
SHAPE_C,237,0,237,237,237,237,237,237,231,237,237,237,237,237
SHAPE_L,415,0,415,415,415,415,415,415,384,415,415,415,415,415
SHAPE_MC,31,0,31,31,31,31,31,31,31,31,31,31,31,31
SQUARE,152,0,152,152,152,152,152,152,139,152,152,152,152,152


### New Inventory Data (dating back 2016)

In [52]:
inv_hist2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779395 entries, 0 to 779394
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   CalendarDay      779395 non-null  datetime64[ns]
 1   Plant            779395 non-null  int64         
 2   Material         779395 non-null  int64         
 3   PH3              779395 non-null  object        
 4   PH5              779395 non-null  object        
 5   PH               779395 non-null  object        
 6   Material type    779395 non-null  object        
 7   Tons             779395 non-null  float64       
 8   MATNR            779395 non-null  int64         
 9   MAKTX            779395 non-null  object        
 10  Shape            776588 non-null  object        
 11  SizeForQualCert  776588 non-null  object        
 12  MTO_MTS          771934 non-null  object        
 13  Planning_Group   776585 non-null  object        
 14  PTP_Group        776

In [53]:
clean_columns(inv_hist2) 

Index(['calendar_day', 'plant', 'material', 'ph3', 'ph5', 'ph',
       'material_type', 'tons', 'matnr', 'maktx', 'shape',
       'size_for_qual_cert', 'mto_mts', 'planning_group', 'ptp_group',
       'mill_combo_code', 'pg_group_code', 'block_resource', 'l5code',
       'steel_grade', 'length_imperial', 'length_text', 'weight_per_piece'],
      dtype='object')

In [54]:
inv_reduced2 = inv_hist2[["calendar_day","material", "maktx","material_type", "mto_mts", "ph","shape", "block_resource",
                          "size_for_qual_cert", "steel_grade","length_text", "tons"]]
inv_reduced2.rename(columns={"material":"sku", "maktx":"material_description", "mto_mts":"strategy", "block_resource":"block", 
                             "size_for_qual_cert":"size"}, inplace=True)
inv_reduced2 = inv_reduced2.sort_values(["calendar_day", "material_description"])
inv_reduced2.head()

,calendar_day,sku,material_description,material_type,strategy,ph,shape,block,size,steel_grade,length_text,tons
0,2016-01-03,102003236,"BL 5 1/8X7 3/8 1010M-446_1302 3,800",ZERT,MTO,LS-Mills-Bil-Bil-Bil-Billets,RECTANGULAR,NaN,5 1/8 X 7 3/8,1010M-446_1302,0,7.600
16,2016-01-03,106020238,"C 10X15.3# GGMULTI 20'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"20'00""",200.736
17,2016-01-03,106020239,"C 10X15.3# GGMULTI 40'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"40'00""",352.206
18,2016-01-03,106020242,"C 10X15.3# GGMULTI 50'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"50'00""",99.450
19,2016-01-03,106020243,"C 10X15.3# GGMULTI 60'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"60'00""",26.622


In [55]:
#changing dtype
inv_reduced2["strategy"] = inv_reduced2["strategy"].astype("category")
inv_reduced2["sku"] = inv_reduced2["sku"].astype("object")

In [56]:
#removing non-end use product hierarchy
inv_reduced2 = inv_reduced2[(inv_reduced2["ph"]!= "LS-Mills-Bil-Bil-Bil-Billets") & 
                            (inv_reduced2["ph"]!= "DO NOT USE") &
                            (inv_reduced2["ph"]!= "LS-Mills-Mer-Mer-MBQ-MBQ HRB")]
inv_reduced2["ph"].value_counts()

LS-Mills-Mer-Mer-Strc A&C-Angles          263153
LS-Mills-Mer-Mer-Lght Shp-MBQ Angles      174981
LS-Mills-Mer-Mer-Strc A&C-Channels        164836
LS-Mills-Mer-Mer-MBQ-MBQ Fl                70798
LS-Mills-Reb-Reb-Str Reb-Str Reb           41162
LS-Mills-Mer-Mer-Lght Shp-MBQ Channels     37024
LS-Mills-Reb-Reb-Reb Spc-Rebar MRB          9864
LS-Mills-Mer-Mer-Strc A&C-MC Channels       6594
LS-Mills-Mer-Mer-MBQ-MBQ Rd                 3091
LS-Mills-Mer-Mer-MBQ-MBQ Sq                 1748
LS-Mills-Mer-Mer-MBQ-Pencil Rod              895
AL-Laminaci√≥n-Perfiles-Canal                277
SS-Mills-SBQ-SBQ-SBQ-SBQ Rd                  170
Name: ph, dtype: int64

In [57]:
inv_reduced2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774593 entries, 16 to 779020
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   calendar_day          774593 non-null  datetime64[ns]
 1   sku                   774593 non-null  object        
 2   material_description  774593 non-null  object        
 3   material_type         774593 non-null  object        
 4   strategy              767339 non-null  category      
 5   ph                    774593 non-null  object        
 6   shape                 771786 non-null  object        
 7   block                 769246 non-null  object        
 8   size                  771786 non-null  object        
 9   steel_grade           771786 non-null  object        
 10  length_text           769268 non-null  object        
 11  tons                  774593 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(9)


### Sales Order Data

In [58]:
clean_columns(order_hist)

Index(['sold_to', 'ship_to', 'ship_to_city', 'ship_to_state', 'order_no',
       'item_no', 'material_number', 'material_description', 'division',
       'reject_reason_code', 'po_line_item_no', 'cust_material_no',
       'overdeliv_tol', 'underdeliv_tol', 'plant', 'plant_name', 'storage_loc',
       'shipping_point', 'route', 'sales_district_code', 'requested_date',
       'confirmed_date', 'order_qty_sales_units', 'confirmed_qty', 'sales_uom',
       'weight_per_piece', 'req_qty_base_uom', 'base_uom', 'transp_plan_date',
       'material_avail_date', 'load_date', 'plan_goods_issue_date',
       'delivery_block', 'qty_on_delivery', 'qty_on_shipment', 'qty_shipped',
       'qty_on_delivery_sales_units', 'qty_on_shipment_sales_units',
       'qty_shipped_sales_units', 'item_status'],
      dtype='object')

In [59]:
order_hist.head()

,sold_to,ship_to,ship_to_city,ship_to_state,order_no,item_no,material_number,material_description,division,reject_reason_code,...,load_date,plan_goods_issue_date,delivery_block,qty_on_delivery,qty_on_shipment,qty_shipped,qty_on_delivery_sales_units,qty_on_shipment_sales_units,qty_shipped_sales_units,item_status
0,100091881,100103709,CANDIAC,QC,171122,60,106020877,"L 6X6X1/2 A36/44W/A529-50 40'00""",5,,...,2013-02-06,2013-02-07,,0.0,0.0,583296.000,0.0,0.0,82176.000,Completely Processed
1,100091881,100103709,CANDIAC,QC,171122,90,106020931,"L 6X6X3/8 50W 46'05""",5,,...,2013-03-27,2013-03-28,,0.0,0.0,329222.544,0.0,0.0,116504.016,Completely Processed
2,100091881,100103709,CANDIAC,QC,171122,50,106020931,"L 6X6X3/8 50W 46'05""",5,,...,2012-12-14,2012-12-17,,0.0,0.0,91296.000,0.0,0.0,91296.000,Completely Processed
3,100092158,100099545,NISKU,AB,358536,30,106021399,"C 6X8.2# A36/44W/A529-50 40'00""",3,,...,2013-06-03,2013-06-18,,0.0,0.0,59040.000,0.0,0.0,180.000,Completely Processed
4,100092158,100099545,NISKU,AB,358536,50,106021716,"C 8X11.5# GGMULTI 40'00""",3,,...,2013-06-07,2013-06-24,,0.0,0.0,18400.000,0.0,0.0,40.000,Completely Processed


In [60]:
#Reducing main data frame
order_reduced = order_hist[["requested_date",'confirmed_date','material_avail_date', 'load_date', 'plan_goods_issue_date','sales_district_code',
                            "division", "sold_to", "ship_to","route","ship_to_city", "ship_to_state", "material_number",'material_description',
                            'order_qty_sales_units', 'confirmed_qty', 'sales_uom','qty_on_shipment','qty_shipped_sales_units', 'item_status',]]
order_reduced.rename(columns={"material_number":"sku"}, inplace=True)
order_reduced = order_reduced.sort_values(["confirmed_date", "material_description"])
order_reduced.head(-5)

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,ship_to_city,ship_to_state,sku,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status
188922,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,MISSISSAUGA,ON,106029006,"L 2X2X3/8 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed
188915,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,MISSISSAUGA,ON,106028705,"L 4X3X3/8 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed
188916,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,MISSISSAUGA,ON,106028755,"L 4X4X1/4 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed
1497,2012-07-22,2012-07-22,2013-07-19,2013-07-19,2013-07-22,,5,100061219,100061228,C00333,CAMBRIDGE,ON,106022700,"L 5X3 1/2X5/16 A36/44W/A529-50 40'00""",10.44,10.44,TON,0.0,20.88,Completely Processed
119,2012-10-01,2012-10-01,2012-10-26,2012-10-26,2012-10-29,,5,100091890,100094606,C00427,KITCHENER,ON,106020434,"C 3X5# A36/44W/A529-50 20'00""",4800.00,4800.00,LB,0.0,9600.00,Completely Processed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186380,1900-01-01,2020-09-08,2020-09-03,2020-09-03,2020-09-04,100311,3,100061219,100061228,C00333,CAMBRIDGE,ON,109008606,"F 1X7 GGMULTI 20'00""",9520.00,9520.00,LB,NaN,NaN,Not Yet Processed
187076,2020-04-01,2020-09-08,2020-09-03,2020-09-03,2020-09-04,100311,3,100092135,100095263,C07102,SAINT-BRUNO,QC,109008606,"F 1X7 GGMULTI 20'00""",19040.00,19040.00,LB,NaN,NaN,Not Yet Processed
185362,2020-09-08,2020-09-08,2020-09-03,2020-09-03,2020-09-04,100313,3,100092191,100095443,C00486,MISSISSAUGA,ON,106022550,"L 2 1/2X2 1/2X1/4 55W 60'00""",103320.00,103320.00,LB,NaN,NaN,Not Yet Processed
183265,2020-09-11,2020-09-11,2020-09-09,2020-09-09,2020-09-10,100313,3,100092191,100095443,C00486,MISSISSAUGA,ON,106022472,"L 2X2X1/4 55W 60'00""",93600.00,93600.00,LB,NaN,NaN,Not Yet Processed


In [61]:
order_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188953 entries, 188922 to 177013
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   requested_date           188953 non-null  datetime64[ns]
 1   confirmed_date           188953 non-null  datetime64[ns]
 2   material_avail_date      188953 non-null  datetime64[ns]
 3   load_date                188953 non-null  datetime64[ns]
 4   plan_goods_issue_date    188953 non-null  datetime64[ns]
 5   sales_district_code      188953 non-null  object        
 6   division                 188953 non-null  int64         
 7   sold_to                  188953 non-null  int64         
 8   ship_to                  188953 non-null  int64         
 9   route                    188953 non-null  object        
 10  ship_to_city             188953 non-null  object        
 11  ship_to_state            188953 non-null  object        
 12  sku        

In [62]:
#Change type of sku
order_reduced["sku"] =order_reduced["sku"].astype("object")

#### Understanding Orders not confirmed for requested date == 1900-01-01

In [63]:
# Confirned date only has three of these, safe to use. 
order_reduced[order_reduced["confirmed_date"]=='1900-01-01'].shape

(3, 20)

In [64]:
#Most are completely processed, but with nearly 1/5 on-process
order_reduced[order_reduced["requested_date"]=='1900-01-01']["item_status"].value_counts()

Completely Processed    4447
Not Yet Processed        912
Partially Processed      155
Name: item_status, dtype: int64

In [65]:
#Most are from late 2019 - early 2020
order_reduced[order_reduced["requested_date"]=='1900-01-01'].sample(10)

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,ship_to_city,ship_to_state,sku,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status
174748,1900-01-01,2020-04-03,2020-04-01,2020-04-01,2020-04-02,100306,3,100091637,100099292,U08596,WAUSAU,WI,106027225,"C 4X5.4# GGMULTI 40'00""",15552.0,15552.0,LB,0.0,15552.0,Completely Processed
164002,1900-01-01,2019-10-06,2019-10-06,2019-10-06,2019-10-06,100311,3,100092146,100103991,999999,NIAGARA FALLS,ON,109008106,"F 3/8X8 GGMULTI 20'00""",12252.0,12252.0,LB,0.0,24504.0,Completely Processed
179922,1900-01-01,2020-05-15,2020-04-13,2020-04-13,2020-04-28,100312,3,100092155,100987501,CM0801,CORMAN PARK,SK,106028782,"L 2 1/2X2 1/2X1/4 GGMULTI 20'00""",12628.0,12628.0,LB,0.0,0.0,Completely Processed
171806,1900-01-01,2019-11-05,2019-11-01,2019-11-01,2019-11-04,100209,2,100092147,100114222,C07206,ETOBICOKE,ON,110001621,RB 30M 400W 18M 00CM,82413.0,82413.0,LB,0.0,174100.0,Completely Processed
187928,1900-01-01,2020-07-08,2020-07-06,2020-07-06,2020-07-07,100302,3,100092247,100095602,C00752,DRUMMONDVILLE,QC,106028974,"L 3 1/2X3 1/2X5/16 GGMULTI 40'00""",41472.0,41472.0,LB,NaN,NaN,Not Yet Processed
178819,1900-01-01,2020-06-03,2020-06-03,2020-06-03,2020-06-03,100311,3,100051178,100103861,999999,SAINT-JEROME,QC,106020529,"L 6X4X3/8 MERSPC 40'00""",49200.0,49200.0,LB,59040.0,0.0,Partially Processed
182900,1900-01-01,2020-07-14,2020-07-10,2020-07-10,2020-07-13,100209,2,100073247,100095381,C00427,KITCHENER,ON,110001535,RB 15M 400W 18M 00CM,448550.0,448550.0,LB,0.0,171940.0,Partially Processed
184014,1900-01-01,2020-04-24,2020-04-20,2020-04-20,2020-04-21,100311,3,100061270,100061277,C00236,SAINT JOHN,NB,106020239,"C 10X15.3# GGMULTI 40'00""",19584.0,19584.0,LB,0.0,0.0,Completely Processed
174306,1900-01-01,2020-01-01,2019-12-27,2019-12-27,2019-12-30,100311,3,100051178,100103819,C00764,JOLIETTE,QC,106028496,"L 4X3X1/4 GGMULTI 20'00""",12992.0,12992.0,LB,0.0,25984.0,Completely Processed
181304,1900-01-01,2020-03-02,2020-02-27,2020-02-27,2020-02-28,100209,2,100092177,100095403,C00970,VICTORIAVILLE,QC,110001535,RB 15M 400W 18M 00CM,1435360.0,1435360.0,LB,0.0,2758580.0,Completely Processed


#### Explore Units of Measure

In [66]:
order_reduced["sales_uom"].value_counts()

LB     182513
TON      6364
ST         62
ZNT        11
            3
Name: sales_uom, dtype: int64

In [67]:
order_reduced[order_reduced["sales_uom"]=="TON"].sample(3)

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,ship_to_city,ship_to_state,sku,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status
41603,2014-07-31,2014-08-04,2014-07-29,2014-07-29,2014-07-30,,3,100060909,100060960,U05194,WILLMAR,MN,106028666,"L 5X3 1/2X1/2 GGMULTI 20'00""",5.44,5.44,TON,0.0,10.88,Completely Processed
22071,2013-11-01,2013-11-01,2013-10-29,2013-10-29,2013-10-30,,3,100073186,100104393,U02974,MABLETON,GA,106022002,"L 3X2X1/2 GGMULTI 40'00""",4.62,4.62,TON,0.0,9.24,Completely Processed
6152,2013-01-31,2013-01-31,2013-01-29,2013-01-29,2013-01-30,,2,100092467,100095997,C00895,SAINT-EUSTACHE,QC,110001535,RB 15M 400W 18M 00CM,40.00,40.00,TON,0.0,78.45,Completely Processed


#### Convert LB to Tons

In [68]:
if "LB" in order_reduced["sales_uom"].unique():
    order_reduced.loc[order_reduced["sales_uom"] == "LB", "confirmed_qty"] = order_reduced["confirmed_qty"].apply(lambda x: x / 2000)
    order_reduced.loc[order_reduced["sales_uom"] == "LB", "order_qty_sales_units"] = order_reduced["order_qty_sales_units"].apply(lambda x: x / 2000)
    order_reduced.loc[order_reduced["sales_uom"] == "LB", "qty_on_shipment"] = order_reduced["qty_on_shipment"].apply(lambda x: x / 2000)
    order_reduced.loc[order_reduced["sales_uom"] == "LB", "qty_shipped_sales_units"] = order_reduced["qty_shipped_sales_units"].apply(lambda x: x / 2000)
    order_reduced.loc[order_reduced["sales_uom"] == "LB", "sales_uom"] = "TON"

#### Add Unique Size, Block, Hierarchy to Sales Data

In [69]:
inv_reduced2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774593 entries, 16 to 779020
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   calendar_day          774593 non-null  datetime64[ns]
 1   sku                   774593 non-null  object        
 2   material_description  774593 non-null  object        
 3   material_type         774593 non-null  object        
 4   strategy              767339 non-null  category      
 5   ph                    774593 non-null  object        
 6   shape                 771786 non-null  object        
 7   block                 769246 non-null  object        
 8   size                  771786 non-null  object        
 9   steel_grade           771786 non-null  object        
 10  length_text           769268 non-null  object        
 11  tons                  774593 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(9)


In [70]:
size_dict = inv_reduced2.groupby(["sku","size"]).sum()
size_dict = size_dict.reset_index()
size_dict = size_dict.drop("tons", axis=1)

In [71]:
block_dict = inv_reduced2.groupby(["sku","block"]).sum()
block_dict = block_dict.reset_index()
block_dict = block_dict.drop("tons", axis=1)

In [72]:
ph_dict = inv_reduced2.groupby(["sku","ph"]).sum()
ph_dict = ph_dict.reset_index()
ph_dict = ph_dict.drop("tons", axis=1)

In [73]:
block_dict.sample(5)

,sku,block
88,106020352,"4"" CHN"
709,106035951,"4"" CHN"
1303,106302917,"2"" ANG"
1644,110001613,10MM
290,106022607,"2-1/2"" ANG"


In [74]:
order_reduced = order_reduced.merge(size_dict, how="left", left_on="sku", right_on="sku")
order_reduced = order_reduced.merge(block_dict, how="left", left_on="sku", right_on="sku")
order_reduced = order_reduced.merge(ph_dict, how="left", left_on="sku", right_on="sku")

In [75]:
order_reduced.columns

Index(['requested_date', 'confirmed_date', 'material_avail_date', 'load_date',
       'plan_goods_issue_date', 'sales_district_code', 'division', 'sold_to',
       'ship_to', 'route', 'ship_to_city', 'ship_to_state', 'sku',
       'material_description', 'order_qty_sales_units', 'confirmed_qty',
       'sales_uom', 'qty_on_shipment', 'qty_shipped_sales_units',
       'item_status', 'size', 'block', 'ph'],
      dtype='object')

In [76]:
order_reduced.head()

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,...,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status,size,block,ph
0,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,...,"L 2X2X3/8 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed,2X2X3/8,"2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles
1,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,...,"L 4X3X3/8 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed,4X3X3/8,"4 X 3"" ANG",LS-Mills-Mer-Mer-Strc A&C-Angles
2,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,100311,3,100091887,100094602,C00486,...,"L 4X4X1/4 GGMULTI 20'00""",0.00,0.00,,NaN,NaN,Not Yet Processed,4X4X1/4,"4"" ANG",LS-Mills-Mer-Mer-Strc A&C-Angles
3,2012-07-22,2012-07-22,2013-07-19,2013-07-19,2013-07-22,,5,100061219,100061228,C00333,...,"L 5X3 1/2X5/16 A36/44W/A529-50 40'00""",10.44,10.44,TON,0.0,20.88,Completely Processed,NaN,NaN,NaN
4,2012-10-01,2012-10-01,2012-10-26,2012-10-26,2012-10-29,,5,100091890,100094606,C00427,...,"C 3X5# A36/44W/A529-50 20'00""",2.40,2.40,TON,0.0,4.80,Completely Processed,NaN,NaN,NaN


In [77]:
order_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188983 entries, 0 to 188982
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   requested_date           188983 non-null  datetime64[ns]
 1   confirmed_date           188983 non-null  datetime64[ns]
 2   material_avail_date      188983 non-null  datetime64[ns]
 3   load_date                188983 non-null  datetime64[ns]
 4   plan_goods_issue_date    188983 non-null  datetime64[ns]
 5   sales_district_code      188983 non-null  object        
 6   division                 188983 non-null  int64         
 7   sold_to                  188983 non-null  int64         
 8   ship_to                  188983 non-null  int64         
 9   route                    188983 non-null  object        
 10  ship_to_city             188983 non-null  object        
 11  ship_to_state            188983 non-null  object        
 12  sku             

In [78]:
order_reduced[order_reduced["size"].isna()].sample(15)

,requested_date,confirmed_date,material_avail_date,load_date,plan_goods_issue_date,sales_district_code,division,sold_to,ship_to,route,...,material_description,order_qty_sales_units,confirmed_qty,sales_uom,qty_on_shipment,qty_shipped_sales_units,item_status,size,block,ph
3397,2012-11-21,2012-11-28,2012-11-26,2012-11-26,2012-11-27,,2,100051178,100114541,C00861,...,RB 15M 400W 12M 65CM,35.000,35.000,TON,0.0,67.320,Completely Processed,NaN,NaN,NaN
42134,2014-09-15,2014-09-15,2014-09-15,2014-09-15,2014-09-15,,3,100060551,100060565,999999,...,"L 4X3X1/4 A36/44W/A529-50 20'00""",3.074,3.074,TON,0.0,6.148,Completely Processed,NaN,NaN,NaN
10141,2013-03-20,2013-04-02,2013-03-28,2013-03-28,2013-04-01,,3,100060551,100060567,C00722,...,L 2 1/2X2 1/2X5/16 A36/44W/A529-50 20'00,6.300,6.300,TON,0.0,12.500,Completely Processed,NaN,NaN,NaN
8947,2013-01-31,2013-03-13,2013-02-08,2013-02-08,2013-02-26,,5,100061174,100061186,CR0037,...,"L 4X4X3/8 A36/44W/A529-50 20'00""",11.760,11.760,TON,0.0,23.520,Completely Processed,NaN,NaN,NaN
27606,2014-01-30,2014-01-30,2014-01-30,2014-01-30,2014-01-30,,3,100091983,100336810,999999,...,"L 2 1/2X2 1/2X1/4 A36/44W/A529-50 20'00""",3.157,3.157,TON,0.0,6.314,Completely Processed,NaN,NaN,NaN
10028,2013-03-22,2013-04-01,2013-03-27,2013-03-27,2013-03-28,,3,100091983,100099290,C00304,...,"C 5X9# A36/44W/A529-50 20'00""",3.240,3.240,TON,0.0,6.480,Completely Processed,NaN,NaN,NaN
97,2012-10-09,2012-10-09,2012-12-03,2012-12-03,2012-12-04,,5,100061219,100061228,C00333,...,"L 5X5X5/8 A36/44W/A529-50 20'00""",2.000,2.000,TON,0.0,4.000,Completely Processed,NaN,NaN,NaN
3814,2012-12-02,2012-12-06,2012-12-04,2012-12-04,2012-12-05,,5,100091899,100094637,C00786,...,"C 4X5.4# A36/44W/A529-50 40'00""",11.664,11.664,TON,0.0,23.328,Completely Processed,NaN,NaN,NaN
35024,2014-05-23,2014-05-27,2014-05-21,2014-05-21,2014-05-22,,3,100091998,100094880,U04599,...,"C 6X10.5# A36/44W/A529-50 40'00""",5.040,5.040,TON,0.0,10.080,Completely Processed,NaN,NaN,NaN
9128,2013-03-15,2013-03-15,2013-03-08,2013-03-08,2013-03-11,,5,100061247,100061261,C00253,...,"L 5X3 1/2X3/8 A36/44W/A529-50 20'00""",6.240,6.240,TON,0.0,12.480,Completely Processed,NaN,NaN,NaN


# Saving Reduced Datasets

In [79]:
data_reduced = [ship_reduced, inv_reduced, atp_reduced, prod_reduced, order_reduced, inv_reduced2]

In [80]:
#storing cleaned Prod
%store prod_reduced

#storing cleaned Ship
%store ship_reduced

#storing cleaned Inventory
%store inv_reduced

#storing cleaned ATP
%store atp_reduced

#storing orders
%store order_reduced

#storing new(2016) inventory
%store inv_reduced2

Stored 'prod_reduced' (DataFrame)
Stored 'ship_reduced' (DataFrame)
Stored 'inv_reduced' (DataFrame)
Stored 'atp_reduced' (DataFrame)
Stored 'order_reduced' (DataFrame)
Stored 'inv_reduced2' (DataFrame)


# Single Merged Dataframe 

## Create Master Table: [Date, Sku], Size, Block, Shape, PH

In [81]:
inv_ind = inv_reduced2.groupby(["calendar_day","sku"]).sum()
inv_ind = inv_ind.reset_index()
inv_ind = inv_ind.drop("tons",axis=1)
inv_ind

,calendar_day,sku
0,2016-01-03,106019137
1,2016-01-03,106019140
2,2016-01-03,106019186
3,2016-01-03,106019248
4,2016-01-03,106019507
...,...,...
774588,2020-03-24,106029590
774589,2020-03-24,106029602
774590,2020-03-24,106029604
774591,2020-03-24,106029606


In [82]:
order_ind = order_reduced.groupby(["confirmed_date","sku"]).sum()
order_ind = order_ind.reset_index()
order_ind = order_ind.drop(["sold_to","ship_to","division","order_qty_sales_units",
                            "confirmed_qty","qty_on_shipment","qty_shipped_sales_units"], axis=1)
order_ind = order_ind.rename(columns={"confirmed_date":"calendar_day"}) 
order_ind

,calendar_day,sku
0,1900-01-01,106028705
1,1900-01-01,106028755
2,1900-01-01,106029006
3,2012-07-22,106022700
4,2012-10-01,106020353
...,...,...
127180,2020-11-10,109004177
127181,2020-11-10,109005409
127182,2020-12-15,106021760
127183,2020-12-15,109005461


In [83]:
ship_ind = ship_reduced.groupby(["shipping_day","sku"]).sum()
ship_ind = ship_ind.reset_index()
ship_ind = ship_ind.drop(["shipment_tons"], axis=1)
ship_ind = ship_ind.rename(columns ={"shipping_day":"calendar_day"})
ship_ind

,calendar_day,sku
0,2015-01-02,106021714
1,2015-01-02,106023056
2,2015-01-02,106023082
3,2015-01-02,106029532
4,2015-01-02,106029546
...,...,...
107715,2020-02-21,110001504
107716,2020-02-21,110001529
107717,2020-02-21,110001535
107718,2020-02-21,110001774


In [84]:
prod_ind = prod_reduced.groupby(["calendar_day","sku"]).sum()
prod_ind = prod_ind.reset_index()
prod_ind = prod_ind.drop("ton", axis=1)
prod_ind

,calendar_day,sku
0,2013-02-04,106023265
1,2013-02-04,110001535
2,2013-02-05,106022739
3,2013-02-05,106022740
4,2013-02-05,106022806
...,...,...
14712,2019-12-30,106029479
14713,2019-12-30,106029521
14714,2019-12-30,106029546
14715,2019-12-30,106301220


In [85]:
master_index = pd.concat([inv_ind,order_ind,ship_ind,prod_ind],axis=0)
master_index = master_index.drop_duplicates()
master_index.sort_values(["calendar_day","sku"])

,calendar_day,sku
0,1900-01-01,106028705
1,1900-01-01,106028755
2,1900-01-01,106029006
3,2012-07-22,106022700
4,2012-10-01,106020353
...,...,...
127180,2020-11-10,109004177
127181,2020-11-10,109005409
127182,2020-12-15,106021760
127183,2020-12-15,109005461


In [86]:
size_dict["sku"].size

1744

In [87]:
master = master_index.merge(size_dict,how="left", on="sku", validate="m:1")

In [88]:
master = master.merge(block_dict,how="left", on="sku", validate="m:1")

In [89]:
master.duplicated(["calendar_day","sku"]).value_counts()

False    880348
dtype: int64

In [90]:
ph_dict[ph_dict.duplicated("sku")]

,sku,ph
222,106022047,LS-Mills-Mer-Mer-Strc A&C-Angles
357,106023171,LS-Mills-Mer-Mer-Strc A&C-Angles
815,106042295,LS-Mills-Mer-Mer-Strc A&C-Angles
875,106043976,LS-Mills-Mer-Mer-Lght Shp-MBQ Channels
909,106044969,LS-Mills-Mer-Mer-Strc A&C-Channels
912,106044972,LS-Mills-Mer-Mer-Strc A&C-Channels
948,106045376,LS-Mills-Mer-Mer-Strc A&C-Channels
1014,106250717,LS-Mills-Mer-Mer-Strc A&C-Channels
1016,106250720,LS-Mills-Mer-Mer-Strc A&C-Channels
1043,106251081,LS-Mills-Mer-Mer-Strc A&C-Channels


In [91]:
ph_dict.shape

(1759, 2)

In [92]:
ph_dict = ph_dict.drop_duplicates("sku", keep="first")

In [93]:
master = master.merge(ph_dict,how="left", on="sku", validate="m:1")

In [94]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 880348 entries, 0 to 880347
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   calendar_day  880348 non-null  datetime64[ns]
 1   sku           880348 non-null  int64         
 2   size          861229 non-null  object        
 3   block         856629 non-null  object        
 4   ph            864196 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 40.3+ MB


In [95]:
#none before 2016
date_limit = pd.Timestamp(2016,1,1)
master = master[master["calendar_day"]>=date_limit]
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 880347
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   calendar_day  814243 non-null  datetime64[ns]
 1   sku           814243 non-null  int64         
 2   size          811071 non-null  object        
 3   block         807619 non-null  object        
 4   ph            813878 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 37.3+ MB


In [96]:
# #none after 2020-3-15
# date_limit = pd.Timestamp(2020,3,15)
# master = master[master["calendar_day"]<=date_limit]
# master.info()

In [97]:
master[master.isna().any(axis=1)]

,calendar_day,sku,size,block,ph
89,2016-01-03,106022528,NaN,"2-1/2"" ANG",LS-Mills-Mer-Mer-Lght Shp-MBQ Angles
138,2016-01-03,106027184,3 X 5#,NaN,LS-Mills-Mer-Mer-Lght Shp-MBQ Channels
146,2016-01-03,106027221,4 X 5.4#,NaN,LS-Mills-Mer-Mer-Strc A&C-Channels
148,2016-01-03,106027284,3 X 4.1#,NaN,LS-Mills-Mer-Mer-Lght Shp-MBQ Channels
351,2016-01-03,110001450,35M,NaN,LS-Mills-Reb-Reb-Str Reb-Str Reb
...,...,...,...,...,...
879976,2019-05-26,110001450,35M,NaN,LS-Mills-Reb-Reb-Str Reb-Str Reb
880089,2019-08-01,106030348,NaN,NaN,NaN
880142,2019-09-13,110001450,35M,NaN,LS-Mills-Reb-Reb-Str Reb-Str Reb
880143,2019-09-14,110001450,35M,NaN,LS-Mills-Reb-Reb-Str Reb-Str Reb


## Inventory

### Checking Nulls

In [98]:
#creating dataframe for analysis of nulls in inventory
cols = ["shape", "block", "size"]

nulls = inv_reduced2[inv_reduced2[cols[2]].isna()]

In [99]:
#negligible by size
nulls.sum()

sku                                                          297620592805
material_description    L 2 1/2X2 1/2X3/16 A36/44W/A529-50 20'00L 2 1/...
material_type           ZERTZERTZERTZERTZERTZERTZERTZERTZERTZERTZERTZE...
strategy                MTOMTOMTOMTOMTOMTOMTOMTOMTOMTOMTOMTOMTOMTOMTOM...
ph                      LS-Mills-Mer-Mer-Lght Shp-MBQ AnglesLS-Mills-M...
shape                                                                   0
block                   2-1/2" ANG2-1/2" ANG2-1/2" ANG2-1/2" ANG2-1/2"...
size                                                                    0
steel_grade                                                             0
length_text                                                             0
tons                                                               26.573
dtype: object

### Dimensions of Skus, Sizes, Blocks, Hierchies & Shapes

In [100]:
sku = inv_reduced2["sku"].nunique()
size = inv_reduced2["size"].nunique()
block = inv_reduced2["block"].nunique()
ph = inv_reduced2["ph"].nunique()
shape = inv_reduced2["shape"].nunique()

print(f"Unique Dimensions \n  Skus: {sku}\n  Sizes: {size}\n  Blocks: {block}\n  Hierarchies: {ph}\n  Shapes:{shape}")

Unique Dimensions 
  Skus: 1747
  Sizes: 314
  Blocks: 112
  Hierarchies: 13
  Shapes:7


In [101]:
inv_reduced2[inv_reduced2["shape"]=="SHAPE_L"]["size"].nunique()

129

In [102]:
# #dimensions
# dimension = ["sku", "size", "block", "ph", "shape"]

# #change dimension index to change dimensions
# columns = ["calendar_day", dimension[0]]

### Adding IN_ prefix

In [103]:
if "IN_" not in inv_reduced2.columns[0]:
    inv_reduced2.columns = ["IN_" + str(col) for col in inv_reduced2.columns]

In [104]:
inv_reduced2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774593 entries, 16 to 779020
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   IN_calendar_day          774593 non-null  datetime64[ns]
 1   IN_sku                   774593 non-null  object        
 2   IN_material_description  774593 non-null  object        
 3   IN_material_type         774593 non-null  object        
 4   IN_strategy              767339 non-null  category      
 5   IN_ph                    774593 non-null  object        
 6   IN_shape                 771786 non-null  object        
 7   IN_block                 769246 non-null  object        
 8   IN_size                  771786 non-null  object        
 9   IN_steel_grade           771786 non-null  object        
 10  IN_length_text           769268 non-null  object        
 11  IN_tons                  774593 non-null  float64       
dtypes: category(1),

## Sales Orders

### Adding SO_ prefix

In [105]:
if "SO_" not in order_reduced.columns[0]:
    order_reduced.columns = ["SO_" + str(col) for col in order_reduced.columns]

In [106]:
order_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188983 entries, 0 to 188982
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   SO_requested_date           188983 non-null  datetime64[ns]
 1   SO_confirmed_date           188983 non-null  datetime64[ns]
 2   SO_material_avail_date      188983 non-null  datetime64[ns]
 3   SO_load_date                188983 non-null  datetime64[ns]
 4   SO_plan_goods_issue_date    188983 non-null  datetime64[ns]
 5   SO_sales_district_code      188983 non-null  object        
 6   SO_division                 188983 non-null  int64         
 7   SO_sold_to                  188983 non-null  int64         
 8   SO_ship_to                  188983 non-null  int64         
 9   SO_route                    188983 non-null  object        
 10  SO_ship_to_city             188983 non-null  object        
 11  SO_ship_to_state            188983 non-

### Filter Data>=2016

In [107]:
date_limit = pd.Timestamp(2016,1,1)
order_reduced = order_reduced[order_reduced["SO_confirmed_date"]>=date_limit]

In [108]:
order_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120159 entries, 68824 to 188982
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   SO_requested_date           120159 non-null  datetime64[ns]
 1   SO_confirmed_date           120159 non-null  datetime64[ns]
 2   SO_material_avail_date      120159 non-null  datetime64[ns]
 3   SO_load_date                120159 non-null  datetime64[ns]
 4   SO_plan_goods_issue_date    120159 non-null  datetime64[ns]
 5   SO_sales_district_code      120159 non-null  object        
 6   SO_division                 120159 non-null  int64         
 7   SO_sold_to                  120159 non-null  int64         
 8   SO_ship_to                  120159 non-null  int64         
 9   SO_route                    120159 non-null  object        
 10  SO_ship_to_city             120159 non-null  object        
 11  SO_ship_to_state            120159 

### Dropping Columns not needed

In [109]:
order_reduced.columns

Index(['SO_requested_date', 'SO_confirmed_date', 'SO_material_avail_date',
       'SO_load_date', 'SO_plan_goods_issue_date', 'SO_sales_district_code',
       'SO_division', 'SO_sold_to', 'SO_ship_to', 'SO_route',
       'SO_ship_to_city', 'SO_ship_to_state', 'SO_sku',
       'SO_material_description', 'SO_order_qty_sales_units',
       'SO_confirmed_qty', 'SO_sales_uom', 'SO_qty_on_shipment',
       'SO_qty_shipped_sales_units', 'SO_item_status', 'SO_size', 'SO_block',
       'SO_ph'],
      dtype='object')

In [110]:
cols = ['SO_requested_date', 'SO_confirmed_date', 'SO_plan_goods_issue_date',
        'SO_sku','SO_material_description', 'SO_order_qty_sales_units',
        'SO_confirmed_qty', 'SO_sales_uom', 'SO_item_status']

order_reduced = order_reduced[cols]
order_reduced.head(5)

,SO_requested_date,SO_confirmed_date,SO_plan_goods_issue_date,SO_sku,SO_material_description,SO_order_qty_sales_units,SO_confirmed_qty,SO_sales_uom,SO_item_status
68824,2015-12-29,2016-01-01,2015-12-17,106022569,"L 2 1/2X2 1/2X5/16 55W 60'00""",315.00,315.00,TON,Completely Processed
68825,2016-01-02,2016-01-02,2015-12-16,109008267,"F 3/4X5 GGMULTI 20'00""",3.06,3.06,TON,Completely Processed
68826,2016-01-02,2016-01-02,2015-12-22,106028573,"L 3X3X1/4 GGMULTI 20'00""",12.74,12.74,TON,Completely Processed
68827,2016-01-02,2016-01-02,2015-12-22,106028446,"L 3X3X3/8 GGMULTI 40'00""",15.12,15.12,TON,Completely Processed
68828,2015-12-26,2016-01-02,2015-12-18,106023056,"L 3X3X5/16 55W 60'00""",27.45,27.45,TON,Completely Processed


## Shipments

### Adding SH_ prefix

In [111]:
if "SH_" not in ship_reduced.columns[0]:
    ship_reduced.columns = ["SH_" + str(col) for col in ship_reduced.columns]

In [112]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156711 entries, 155274 to 21756
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   SH_shipping_day          156711 non-null  datetime64[ns]
 1   SH_block                 153079 non-null  object        
 2   SH_strategy              156700 non-null  object        
 3   SH_shape                 156700 non-null  object        
 4   SH_size                  156700 non-null  object        
 5   SH_div                   156711 non-null  object        
 6   SH_sku                   156711 non-null  object        
 7   SH_material_description  156711 non-null  object        
 8   SH_shipment_tons         156711 non-null  float64       
 9   SH_segment               154025 non-null  object        
 10  SH_sold-to_party         156711 non-null  object        
 11  SH_ship-to_party         156711 non-null  object        
 12  SH_postal_co

### Filter Data>=2016

In [113]:
ship_reduced = ship_reduced[ship_reduced["SH_shipping_day"]>=date_limit]

In [114]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131465 entries, 30378 to 21756
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   SH_shipping_day          131465 non-null  datetime64[ns]
 1   SH_block                 128423 non-null  object        
 2   SH_strategy              131454 non-null  object        
 3   SH_shape                 131454 non-null  object        
 4   SH_size                  131454 non-null  object        
 5   SH_div                   131465 non-null  object        
 6   SH_sku                   131465 non-null  object        
 7   SH_material_description  131465 non-null  object        
 8   SH_shipment_tons         131465 non-null  float64       
 9   SH_segment               128799 non-null  object        
 10  SH_sold-to_party         131465 non-null  object        
 11  SH_ship-to_party         131465 non-null  object        
 12  SH_postal_cod

## Production

### Adding PR_ prefix

In [115]:
if "PR_" not in prod_reduced.columns[0]:
    prod_reduced.columns = ["PR_" + str(col) for col in prod_reduced.columns]

In [116]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14820 entries, 192 to 13808
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   PR_calendar_day                    14820 non-null  datetime64[ns]
 1   PR_production_process              14820 non-null  category      
 2   PR_production_process_description  14820 non-null  category      
 3   PR_strategy                        14820 non-null  category      
 4   PR_work_center                     13450 non-null  category      
 5   PR_wc_description                  14820 non-null  category      
 6   PR_block                           14815 non-null  object        
 7   PR_shape                           14795 non-null  category      
 8   PR_size                            14795 non-null  object        
 9   PR_sku                             14820 non-null  object        
 10  PR_material_description         

### Filter Data>=2016

In [117]:
prod_reduced = prod_reduced[prod_reduced["PR_calendar_day"]>=date_limit]

In [118]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9833 entries, 12098 to 13808
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   PR_calendar_day                    9833 non-null   datetime64[ns]
 1   PR_production_process              9833 non-null   category      
 2   PR_production_process_description  9833 non-null   category      
 3   PR_strategy                        9833 non-null   category      
 4   PR_work_center                     8873 non-null   category      
 5   PR_wc_description                  9833 non-null   category      
 6   PR_block                           9828 non-null   object        
 7   PR_shape                           9833 non-null   category      
 8   PR_size                            9833 non-null   object        
 9   PR_sku                             9833 non-null   object        
 10  PR_material_description        

## First Merge (Inv & Order)

In [119]:
date = pd.DataFrame(pd.date_range('1/1/2016','3/15/2020'), columns =["date_index"])
date.head()

,date_index
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05


In [120]:
indexed_inv = date.merge(inv_reduced2, how='left', left_on="date_index", right_on="IN_calendar_day", validate='1:m')
indexed_inv.head(5)

,date_index,IN_calendar_day,IN_sku,IN_material_description,IN_material_type,IN_strategy,IN_ph,IN_shape,IN_block,IN_size,IN_steel_grade,IN_length_text,IN_tons
0,2016-01-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,2016-01-03,106020238,"C 10X15.3# GGMULTI 20'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"20'00""",200.736
3,2016-01-03,2016-01-03,106020239,"C 10X15.3# GGMULTI 40'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"40'00""",352.206
4,2016-01-03,2016-01-03,106020242,"C 10X15.3# GGMULTI 50'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,GGMULTI,"50'00""",99.450


In [121]:
indexed_order = date.merge(order_reduced, how='left', left_on="date_index", right_on="SO_confirmed_date",validate='1:m')

In [122]:
left_cols = ["date_index","IN_sku"]
right_cols = ["SO_confirmed_date", "SO_sku"]

merge1 = indexed_inv.merge(order_reduced, how='outer', left_on=left_cols, right_on=right_cols)

In [123]:
merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 835827 entries, 0 to 835826
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date_index                797164 non-null  datetime64[ns]
 1   IN_calendar_day           796755 non-null  datetime64[ns]
 2   IN_sku                    796755 non-null  object        
 3   IN_material_description   796755 non-null  object        
 4   IN_material_type          796755 non-null  object        
 5   IN_strategy               789867 non-null  category      
 6   IN_ph                     796755 non-null  object        
 7   IN_shape                  793974 non-null  object        
 8   IN_block                  789518 non-null  object        
 9   IN_size                   793974 non-null  object        
 10  IN_steel_grade            793974 non-null  object        
 11  IN_length_text            791473 non-null  object        
 12  IN

In [124]:
merge1.head(5)

,date_index,IN_calendar_day,IN_sku,IN_material_description,IN_material_type,IN_strategy,IN_ph,IN_shape,IN_block,IN_size,...,IN_tons,SO_requested_date,SO_confirmed_date,SO_plan_goods_issue_date,SO_sku,SO_material_description,SO_order_qty_sales_units,SO_confirmed_qty,SO_sales_uom,SO_item_status
0,2016-01-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,2016-01-03,106020238,"C 10X15.3# GGMULTI 20'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,...,200.736,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-03,2016-01-03,106020239,"C 10X15.3# GGMULTI 40'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,...,352.206,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-03,2016-01-03,106020242,"C 10X15.3# GGMULTI 50'00""",ZERT,MTS,LS-Mills-Mer-Mer-Strc A&C-Channels,SHAPE_C,"10"" CHN",10 X 15.3#,...,99.450,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
merge1[(merge1["date_index"].isna()) & (merge1["SO_plan_goods_issue_date"].notna())].sample(20)

,date_index,IN_calendar_day,IN_sku,IN_material_description,IN_material_type,IN_strategy,IN_ph,IN_shape,IN_block,IN_size,...,IN_tons,SO_requested_date,SO_confirmed_date,SO_plan_goods_issue_date,SO_sku,SO_material_description,SO_order_qty_sales_units,SO_confirmed_qty,SO_sales_uom,SO_item_status
798737,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2016-03-18,2016-03-18,2016-03-15,106027353,"C 6X8.2# GGMULTI 40'00""",4.9200,4.9200,TON,Completely Processed
798749,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2016-03-18,2016-03-18,2016-03-18,109008152,"F 3/8X6 GGMULTI 20'00""",8.9580,8.9580,TON,Completely Processed
823129,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2019-08-17,2019-09-27,2019-09-13,109008342,"F 3/8X2 GGMULTI 20'00""",4.9980,4.9980,TON,Completely Processed
831308,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-02-29,2020-05-01,2020-04-15,106028775,"L 2 1/2X2 1/2X1/4 GGMULTI 40'00""",5.1660,5.1660,TON,Completely Processed
830242,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2020-04-01,2020-04-21,2020-04-21,106028454,"L 3X3X5/16 GGMULTI 20'00""",3.0500,3.0500,TON,Not Yet Processed
830365,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1900-01-01,2020-04-23,2020-04-22,106028570,"L 3X3X1/2 GGMULTI 40'00""",9.4000,9.4000,TON,Not Yet Processed
805787,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-03-17,2017-03-17,2017-03-16,106027308,"C 3X4.1# GGMULTI 40'00""",11.8080,11.8080,TON,Completely Processed
804154,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-01-06,2017-01-06,2017-01-06,106028445,"L 3X3X3/8 GGMULTI 20'00""",8.6400,8.6400,TON,Completely Processed
823217,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2019-10-02,2019-10-02,2019-09-17,106028223,"C 7X14.75# GGMULTI 40'00""",18.8800,18.8800,TON,Completely Processed
819213,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2019-04-26,2019-04-26,2019-04-25,106045868,"C 5X6.7# A529-50 CE<.50 42'06""",25.6290,25.6290,TON,Completely Processed


In [126]:
indexed_inv[indexed_inv["IN_sku"] == 106022569].size

14430

In [127]:
indexed_inv[indexed_inv["date_index"] == pd.Timestamp(2016-1-2)].size

0

## Second Attempt

In [128]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 880347
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   calendar_day  814243 non-null  datetime64[ns]
 1   sku           814243 non-null  int64         
 2   size          811071 non-null  object        
 3   block         807619 non-null  object        
 4   ph            813878 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 37.3+ MB


In [129]:
inv_reduced2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774593 entries, 16 to 779020
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   IN_calendar_day          774593 non-null  datetime64[ns]
 1   IN_sku                   774593 non-null  object        
 2   IN_material_description  774593 non-null  object        
 3   IN_material_type         774593 non-null  object        
 4   IN_strategy              767339 non-null  category      
 5   IN_ph                    774593 non-null  object        
 6   IN_shape                 771786 non-null  object        
 7   IN_block                 769246 non-null  object        
 8   IN_size                  771786 non-null  object        
 9   IN_steel_grade           771786 non-null  object        
 10  IN_length_text           769268 non-null  object        
 11  IN_tons                  774593 non-null  float64       
dtypes: category(1),

In [130]:
inv_reduced2.columns

Index(['IN_calendar_day', 'IN_sku', 'IN_material_description',
       'IN_material_type', 'IN_strategy', 'IN_ph', 'IN_shape', 'IN_block',
       'IN_size', 'IN_steel_grade', 'IN_length_text', 'IN_tons'],
      dtype='object')

In [131]:
inv_reduced2 = inv_reduced2[['IN_calendar_day', 'IN_sku', 'IN_material_description',
                            'IN_material_type', 'IN_strategy', 'IN_shape', 'IN_steel_grade',
                            'IN_length_text', 'IN_tons']]

In [132]:
left_cols = ["calendar_day","sku"]
right_cols = ["IN_calendar_day", "IN_sku"]

merge2 = master.merge(inv_reduced2, how="left", left_on=left_cols, right_on=right_cols, validate="1:1")
merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   calendar_day             814243 non-null  datetime64[ns]
 1   sku                      814243 non-null  object        
 2   size                     811071 non-null  object        
 3   block                    807619 non-null  object        
 4   ph                       813878 non-null  object        
 5   IN_calendar_day          774593 non-null  datetime64[ns]
 6   IN_sku                   774593 non-null  object        
 7   IN_material_description  774593 non-null  object        
 8   IN_material_type         774593 non-null  object        
 9   IN_strategy              767339 non-null  category      
 10  IN_shape                 771786 non-null  object        
 11  IN_steel_grade           771786 non-null  object        
 12  IN_length_text  

### Need to aggregate Sales before merging

In [133]:
order_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120159 entries, 68824 to 188982
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SO_requested_date         120159 non-null  datetime64[ns]
 1   SO_confirmed_date         120159 non-null  datetime64[ns]
 2   SO_plan_goods_issue_date  120159 non-null  datetime64[ns]
 3   SO_sku                    120159 non-null  object        
 4   SO_material_description   120159 non-null  object        
 5   SO_order_qty_sales_units  120159 non-null  float64       
 6   SO_confirmed_qty          120159 non-null  float64       
 7   SO_sales_uom              120159 non-null  object        
 8   SO_item_status            120159 non-null  object        
dtypes: datetime64[ns](3), float64(2), object(4)
memory usage: 9.2+ MB


In [134]:
grouped_order = order_reduced.groupby(["SO_confirmed_date", "SO_sku"]).sum()
grouped_order = grouped_order.reset_index()
grouped_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77225 entries, 0 to 77224
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SO_confirmed_date         77225 non-null  datetime64[ns]
 1   SO_sku                    77225 non-null  int64         
 2   SO_order_qty_sales_units  77225 non-null  float64       
 3   SO_confirmed_qty          77225 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 2.4 MB


In [135]:
right_cols = ["SO_confirmed_date", "SO_sku"]

merge3 = merge2.merge(grouped_order, how="left", left_on=left_cols, right_on=right_cols, validate="1:1")
merge3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_calendar_day           774593 non-null  datetime64[ns]
 6   IN_sku                    774593 non-null  object        
 7   IN_material_description   774593 non-null  object        
 8   IN_material_type          774593 non-null  object        
 9   IN_strategy               767339 non-null  category      
 10  IN_shape                  771786 non-null  object        
 11  IN_steel_grade            771786 non-null  object        
 12  IN

### Need to Aggregate Shipment before merge

In [136]:
ship_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131465 entries, 30378 to 21756
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   SH_shipping_day          131465 non-null  datetime64[ns]
 1   SH_block                 128423 non-null  object        
 2   SH_strategy              131454 non-null  object        
 3   SH_shape                 131454 non-null  object        
 4   SH_size                  131454 non-null  object        
 5   SH_div                   131465 non-null  object        
 6   SH_sku                   131465 non-null  object        
 7   SH_material_description  131465 non-null  object        
 8   SH_shipment_tons         131465 non-null  float64       
 9   SH_segment               128799 non-null  object        
 10  SH_sold-to_party         131465 non-null  object        
 11  SH_ship-to_party         131465 non-null  object        
 12  SH_postal_cod

In [137]:
ship_reduced["SH_segment"].value_counts()

NA - DISTRIBUTION     107260
NA - REBAR             14004
NA - CONSTRUCTION       4219
NA - MANUFACTURING      3316
Name: SH_segment, dtype: int64

In [138]:
grouped_ship = ship_reduced.groupby(["SH_shipping_day","SH_sku"]).sum()
grouped_ship = grouped_ship.reset_index()
grouped_ship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89182 entries, 0 to 89181
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   SH_shipping_day   89182 non-null  datetime64[ns]
 1   SH_sku            89182 non-null  int64         
 2   SH_shipment_tons  89182 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.0 MB


In [139]:
right_cols = ["SH_shipping_day", "SH_sku"]

merge4 = merge3.merge(grouped_ship, how="left", left_on=left_cols, right_on=right_cols, validate="1:1")
merge4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_calendar_day           774593 non-null  datetime64[ns]
 6   IN_sku                    774593 non-null  object        
 7   IN_material_description   774593 non-null  object        
 8   IN_material_type          774593 non-null  object        
 9   IN_strategy               767339 non-null  category      
 10  IN_shape                  771786 non-null  object        
 11  IN_steel_grade            771786 non-null  object        
 12  IN

### Explore Prod for duplicates

In [140]:
prod_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9833 entries, 12098 to 13808
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   PR_calendar_day                    9833 non-null   datetime64[ns]
 1   PR_production_process              9833 non-null   category      
 2   PR_production_process_description  9833 non-null   category      
 3   PR_strategy                        9833 non-null   category      
 4   PR_work_center                     8873 non-null   category      
 5   PR_wc_description                  9833 non-null   category      
 6   PR_block                           9828 non-null   object        
 7   PR_shape                           9833 non-null   category      
 8   PR_size                            9833 non-null   object        
 9   PR_sku                             9833 non-null   object        
 10  PR_material_description        

In [141]:
grouped_prod = prod_reduced.groupby(["PR_calendar_day","PR_sku"]).sum()
grouped_prod = grouped_prod.reset_index()
grouped_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9740 entries, 0 to 9739
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PR_calendar_day  9740 non-null   datetime64[ns]
 1   PR_sku           9740 non-null   int64         
 2   PR_ton           9740 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 228.4 KB


In [142]:
right_cols = ["PR_calendar_day", "PR_sku"]

merge5 = merge4.merge(grouped_prod, how="left", left_on=left_cols, right_on=right_cols, validate="1:1")
merge5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_calendar_day           774593 non-null  datetime64[ns]
 6   IN_sku                    774593 non-null  object        
 7   IN_material_description   774593 non-null  object        
 8   IN_material_type          774593 non-null  object        
 9   IN_strategy               767339 non-null  category      
 10  IN_shape                  771786 non-null  object        
 11  IN_steel_grade            771786 non-null  object        
 12  IN

In [143]:
merge5 = merge5.drop(["IN_sku","IN_calendar_day","SO_confirmed_date","SO_sku","SH_shipping_day","SH_sku","PR_calendar_day","PR_sku"], axis=1)

In [144]:
merge5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_material_description   774593 non-null  object        
 6   IN_material_type          774593 non-null  object        
 7   IN_strategy               767339 non-null  category      
 8   IN_shape                  771786 non-null  object        
 9   IN_steel_grade            771786 non-null  object        
 10  IN_length_text            769268 non-null  object        
 11  IN_tons                   774593 non-null  float64       
 12  SO

In [145]:
#saving merged dataframe
#merge5.to_pickle(ser_path + "merged.pkl")

In [148]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 814243 entries, 0 to 814242
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              814243 non-null  datetime64[ns]
 1   sku                       814243 non-null  object        
 2   size                      811071 non-null  object        
 3   block                     807619 non-null  object        
 4   ph                        813878 non-null  object        
 5   IN_material_description   774593 non-null  object        
 6   IN_material_type          774593 non-null  object        
 7   IN_strategy               767339 non-null  category      
 8   IN_shape                  771786 non-null  object        
 9   IN_steel_grade            771786 non-null  object        
 10  IN_length_text            769268 non-null  object        
 11  IN_tons                   774593 non-null  float64       
 12  SO

# Still needs to fill missing holes in sizes, blocks, ph

In [149]:
#Don't know what is happening with nulls! 
merged_df.groupby(["calendar_day","size"]).sum()

IN_tons  SO_order_qty_sales_units  \
calendar_day size                                                  
2016-01-01   2 1/2X2 1/2X5/16      0.0                   315.000   
2016-01-02   3/4 X 5               0.0                     3.060   
             3X3X1/4               0.0                    12.740   
             3X3X3/8               0.0                    15.120   
             3X3X5/16              0.0                    27.450   
...                                ...                       ...   
2020-11-10   1/2 X 9               0.0                     4.284   
             3/8 X 9               0.0                     2.181   
2020-12-15   1 X 10                0.0                     2.040   
             8 X 18.75#            0.0                     4.500   
2020-12-18   4 X 5.4#              0.0                     1.944   

                               SO_confirmed_qty  SH_shipment_tons  PR_ton  
calendar_day size                                                          
2016-01-01   2 1/2X2 1/2X5/16           315.000               0.0     0.0  
2016-01-02   3/4 X 5                      3.060               0.0     0.0  
             3X3X1/4                     12.740               0.0     0.0  
             3X3X3/8                     15.120               0.0     0.0  
             3X3X5/16                    27.450               0.0     0.0  
...                                         ...               ...     ...  
2020-11-10   1/2 X 9                      4.284               0.0     0.0  
             3/8 X 9                      2.181               0.0     0.0  
2020-12-15   1 X 10                       2.040               0.0     0.0  
             8 X 18.75#                   4.500               0.0     0.0  
2020-12-18   4 X 5.4#                     1.944               0.0     0.0  

[242555 rows x 5 columns]